In [4]:
import os
import openai

# .envファイルからAPIキーを読み込む
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [29]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType

from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [40]:
# データの読み込み
df = pd.read_csv('../data/titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [41]:
# データフレームを操作するエージェントの作成
agent = create_pandas_dataframe_agent(
    OpenAI(temperature=0, max_tokens=1000),
    df,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [42]:
text="""
生存したかどうかを、RandomForestで予測してください。
生存したかどうかは、変数Survivedです。1なら生存、0なら死亡です。

入力データセットはdfです。

データセットを訓練用とテスト用に分割してください。データセットの70%を訓練用、30%をテスト用としてください。
訓練用データセットを使って予測モデルを作成し、テスト用データセットを使って予測精度を計算してください。
予測精度は、AUCを計算してください。
"""
agent.run(text)



> Entering new  chain...

Thought: RandomForestを使って予測するためには、まずデータセットを訓練用とテスト用に分割する必要がある。
Action: python_repl_ast
Action Input: from sklearn.model_selection import train_test_split
Observation: 
Thought: 次に、訓練用データセットを使ってRandomForestを作成する。
Action: python_repl_ast
Action Input: from sklearn.ensemble import RandomForestClassifier
Observation: 
Thought: 次に、訓練用データセットを使ってRandomForestを作成する。
Action: python_repl_ast
Action Input: X_train, X_test, y_train, y_test = train_test_split(df.drop('Survived', axis=1), df['Survived'], test_size=0.3, random_state=42)
Observation: 
Thought: 次に、RandomForestを作成する。
Action: python_repl_ast
Action Input: clf = RandomForestClassifier(random_state=42)
Observation: 
Thought: 次に、訓練用データセットを使ってRandomForestを訓練する。
Action: python_repl_ast
Action Input: clf.fit(X_train, y_train)
Observation: ValueError: could not convert string to float: 'Dodge, Master. Washington'
Thought: 文字列をfloatに変換できないので、文字列を含む列を削除する。
Action: python_repl_ast
Action Input: X_train, X_test, y_train, 

'Agent stopped due to iteration limit or time limit.'

In [ ]:
# Iteration limitに達してしまいました。
# 複雑な予測の場合は、コード自体を自動生成させるよりも、自前で用意したツールを利用させたほうが良さそうです。
from langchain.agents import tool

# toolデコレータを使って、関数をツールとして定義。
@tool
def train_and_predict_by_random_forest(df: pd.DataFrame, target_column: str, test_size: float = 0.3) -> float:
    """
    RandomForestの予測モデルを作成し、テストデータに対してyの予測精度のROCを返す関数です。

    Parameters:
    ----------
    df: pd.DataFrame
        訓練用データセット
    target_column: str
        予測対象のカラム名
    test_size: float
        テストデータの割合

    Returns:
    ----------
    roc_auc_score: float
        テストデータに対する予測精度(ROC)
    """
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score

    X = df.drop([target_column], axis=1)
    y = df[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    return roc_auc_score(y_test, y_pred)

In [38]:
# データフレームを操作するエージェントの作成
agent = agent(
    OpenAI(temperature=0),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)